In [1]:
# @title TSYP CHALLENGE
from IPython.display import HTML
html_code = """
<div style="text-align: center;">
    <font color="#ffd700" size="7">
        IEEE CS Tunisia in collaboration with IEEE YP Tunisia
        <br>
        Deep-fake Forensics Challenge
    </font>
</div>
<br>
<div style="text-align: center;">
    <font size="3">
        The challenge revolves around reproducing and enhancing the results presented in a research paper
        entitled:
        "Shallow- and Deep- fake Image Manipulation Localization Using Deep Learning" by Junbin Zhang,
        Hamidreza Tohidypour, Yixiao Wang, and Panos Nasiopoulos.
    </font>
</div>
"""

display(HTML(html_code))

This report will reflect the notebook in the repository which addresses the reproducibility of the research work presented in the **Paper**:
`"Shallow- and Deep-fake Image Manipulation Localization Using Deep Learning" by Junbin Zhang, Hamidreza Tohidypour, Yixiao Wang, and Panos Nasiopoulos.`

## `Introduction`
This report focuses on the critical task of reproducing the research outlined in the paper, exploring the significance of forged image localization and its impact on various aspects of society. The research specifically addresses both "shallowfakes" created with image editing tools and "deepfakes" generated using artificial intelligence techniques. The paper introduces a novel solution capable of effectively localizing manipulated areas in both shallow- and deep-fake images, achieving high inference accuracy.

***The report will be outlined as following :***

## `Datasets, Packages and Libraries Importing`
- Import necessary Python packages and libraries.
- Download The necessary datasets to reproduce the author's work

## `Code Inspection and Performance Assessment`
- Inspect and explain code implementation.
- Evaluate the code from different perspectives, including its ease of implementation, performance characteristics, and its extensibility for future development.

## `Experiments (Author's Work)`
- Conduct the Authour's experiments to validate the proposed solution.
- Apply the solution to various datasets for localization in manipulated areas.
- Present results, visualizations, and comparisons with existing methods.

## `Optimization (Ours)`
- Explore opportunities for solution optimization.
- Apply various optimization techniques to achieve better results.


***In this report, sometimes will include code snippets so we could highlight how we reporduced the author's work and how we optimized on them, with this we will ensure that overall the report will summarize and well present the work we have done.***


# <font color = "#ffd700" size = 6> Datasets, Packages and Libraries Importing
First, we wanted to ensure that we have imported the required Python packages and libraries. Subsequently, we download the essential datasets needed to replicate the author's research.

## `Packages and Libraries Importing`

In [2]:
!pip install gdown --quiet
!pip install ninja --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 6.4 MB/s eta 0:00:00


In [3]:
import os
import sys
import numpy as np
import pandas as pd
import random
from PIL import Image
from IPython.display import display

## `Dataset Downloading`



`The dataset consists of two parts, one with shallowfake
images and one with deepfake images.`
 - **Shallowfakes** : our training dataset is from **CASIAv2**. This dataset includes 7,490 authentic images and 4,948 forged images with slicing and copy-move manipulations, covering different kinds of objects. We split the dataset into training, validation, and test sets at a ratio of 8:1:1. That is to say, at the end we use 5,992 authentic images and 3,958 forged images in training our network. For testing, besides the above-mentioned 10% images from **CASIAv2**, we also included all images from **CASIAv1**, **Columbia**, **COVERAGE**, and **NIST16** in our test set to evaluate the generalizability of our model. In the end, our shallowfake test set includes 1,832 real and 2,259 fake images, covering all three types of shallowfake manipulations (i.e.,
slicing, copy-move, and inpainting).

 - ***DeepFake*** : To address the absence of an image dataset with ground truth masks for manipulated areas in deepfakes, the authors created a dataset using FaceForensics++. This renowned dataset stands out as the sole source offering masks for a significant portion of its deepfake videos. FaceForensics++ comprises 1,363 genuine videos sourced from YouTube, 3,068 videos from Google's DeepFakeDetection dataset, and 5,000 fake videos generated through five distinct automated face manipulation techniques, namely Deepfakes, Face2Face, FaceShifter, FaceSwap, and NeuralTextures.
 <br>
 ---
 ***At this Section we will provide the work we did to download the diffrent datasets. We created some helper functions at first to achieve that. The detailled code will be provided in the notebook, we want to make this report contains less code as much as possible.***

### Helpers
```
Those functions will help us to download the different datasets employed by the authors
```

In [4]:
def download_drive_dataset(link, dataset_name, extention = "zip") :
  """
  Description :
  -------------
    This function will help us to download any dataset from google drive.

  Params:
  ------
    - link : the google drive link of the dataset.
    - dataset_name : the dataset name of the downloaded file and the output folder.
    - extention : Compression Foramt

  """
  file_id = link[link.find("/d/") + 1:].strip('/view').strip("d/")
  file_download_link = "https://docs.google.com/uc?export=download&id=" + file_id

  local_path = "/content/" + dataset_name + f".{extention}"

  !gdown "$file_download_link" -O "$local_path"

  if extention == "zip" :
    path = f"/content/{dataset_name}"
    if not os.path.exists(path):
      os.makedirs(path)
      print(f"Dataset {dataset_name} directort is created !")

    print("Unzipping content...")
    !unzip -qq "$local_path" -d "$path"
  else :
    print("Decompressing method not found !")

In [5]:
def download_dataset_dropbox(link, out_name, out_path) :
  """
  Description :
  -------------
    This function will help us to download any dataset from dropbox.

  Params:
  ------
    - link : the google drive link of the dataset.
    - out_name : the dataset name of the downloaded file.
    - out_path : Folder name which will contain our images after extraction.

  """
  !wget -O  "$out_name" "$link"

  path = f"/content/{out_path}"
  if not os.path.exists(path):
    os.makedirs(path)
    print(f"Dataset {out_path} directort is created !")

  print("Unzipping content...")
  if "zip" in out_name :
    !unzip -qq "$out_name" -d "$path"
  else :
    !tar -xf "/content/$out_name" -C "$path"

### DeepFake Data


In [ ]:
%%time
link = "https://www.dropbox.com/s/o5410tl5v4vxsth/ICNC2023-Deepfakes.tar.xz?dl=0"
out_name = "deepfake.tar.xz"
out_path = "deep-data"

download_dataset_dropbox(link, out_name, out_path)

### ShallowFake Data

```
To train and validate the shallowFake detection model, CASIAv2 will be employed by the authors.
Subsequently, other datasets will be used for testing to evaluate the model's generalization capabilities.
```

#### CASIAv2


In [ ]:
%%time
link = "https://drive.google.com/file/d/1IDUgcoUeonBxx2rASX-_QwV9fhbtqdY8/view"
dataset_name = "casiav2"
download_drive_dataset(link, dataset_name)

***After downloading the CASIAv2 dataset we need to Clone the Masks from CasiaV2 Repo and copy it to the extracted dataset***

In [ ]:
!git clone https://github.com/namtpham/casia2groundtruth
!unzip -qq "/content/casia2groundtruth/CASIA2.0_Groundtruth.zip" -d "/content/casiav2/CASIA2.0_revised/"

#### CASIAv1

In [ ]:
link = "https://drive.google.com/file/d/14f3jU2VsxTYopgSE1Vvv4hMlFvpAKHUY/view"
dataset_name = "casiav1"
download_drive_dataset(link, dataset_name)

***We followed the same approach here for the CASIAv2 dataset, we need to clone the repo to retrieve the ground truth masks***

In [ ]:
!git clone https://github.com/namtpham/casia1groundtruth
!unzip -qq "/content/casia1groundtruth/CASIA 1.0 groundtruth.zip" -d "/content/casiav1/CASIA 1.0 dataset/"

***For this specific Dataset we needed extra steps to follow so we could have our dataset ready. So we extracted the content inside the unzipped downloaded dataset to obtain the Authentic and modified images. We also renamed the datasets folder because spaces will yield to some problems when we try to parse the paths and read our images in training later***

In [11]:
!mv /content/casiav1/CASIA\ 1.0\ dataset/ /content/casiav1/CASIA_1.0_dataset/
!mv /content/casiav1/CASIA_1.0_dataset/CASIA\ 1.0\ groundtruth/ /content/casiav1/CASIA_1.0_dataset/CASIA_1.0_groundtruth/

In [12]:
!unzip -qq /content/casiav1/CASIA_1.0_dataset/Au.zip -d /content/casiav1/CASIA_1.0_dataset/
!unzip -qq /content/casiav1/CASIA_1.0_dataset/Modified\ Tp.zip -d /content/casiav1/CASIA_1.0_dataset/

#### Columbia

***At this step we will download the Columbia Dataset, we will extract the content and we will change the folder layouts of our datasets so we would prevent training problems later***

In [ ]:
%%time
link = "https://www.dropbox.com/sh/786qv3yhvc7s9ki/AACbEEzGPrD3_y38bpWHzgdqa?dl=0"
out_name = "colombia.zip"
out_path = "colombia"

download_dataset_dropbox(link, out_name, out_path)

In [14]:
!rm -rf /content/colombia/4cam_auth
!rm -rf /content/colombia/4cam_splc

In [15]:
!tar -xf "/content/colombia/4cam_auth.tar.bz2" -C "/content/colombia"
!tar -xf "/content/colombia/4cam_splc.tar.bz2" -C "/content/colombia"

In [16]:
!mkdir /content/colombia/masks
!cp -r /content/colombia/4cam_auth/edgemask /content/colombia/masks
!cp -a /content/colombia/4cam_splc/edgemask/. /content/colombia/masks/edgemask/

#### COVERAGE

***Coverage Is from OneDrive so we downloaded it using it's curl command and then we extracted it's content.***

In [17]:
!curl 'https://storage.live.com/downloadfiles/V1/Zip?authKey=%21ADJSupKlX%5FIj8Yc&application=1141147648' \
  -H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7' \
  -H 'Accept-Language: en-US,en;q=0.9' \
  -H 'Cache-Control: max-age=0' \
  -H 'Connection: keep-alive' \
  -H 'Content-Type: application/x-www-form-urlencoded' \
  -H 'Cookie: wlidperf=FR=L&ST=1701337549707; PPLState=1; mkt=en-US; MSPAuth=Disabled; MSPProf=Disabled; NAP=V=1.9&E=1cc3&C=RuGrWk0suTTy6_xP7TiKMuobS4WTJxL6uzp0VQ0IkiHPY8AyMo7c-A&W=1; ANON=A=05C361638D7B64E69AB0999DFFFFFFFF&E=1d1d&W=1; MUID=10FDC95B1BF166B03C2BDA3B1A8667F1; WLSSC=EgArAgMAAAAMgAAAEAABCLNeLPfpwj0mB4EH5zgnyvPE/y9qbT1BZeqIgbUGlCxeMw3154ufCY3Gd+QiUDWOnAOeSgw/CwpqEedi0lX30uhYuFswQzLfVvHxSh8IeF+YF1fNDYLL+AuE6x1oSNnbLJ/iyqMopEIO1ec1PoKVvyHSG26LCwAIBEE/K3rQVuUOvlul4Mp1oAiePk6PGVyjdIwH8ZT6ML6z3YoSMHIm6vMM1W6+igJ4CuEAKqTzOQ8UaceIFaZJHY9v52k+cLK0Oy2ze70DZ6cNQnnbGzHbEzuQMH8bf3XivPnc+sXc3QF4g9iCcTxNFdFBMF4pm/ZpgjwZPusUQg0UJMfY1551LxoBfgAaAf5/AwD9w/o1vOB1Zc5ZaGUQJwAAChCgABAXAGdoYXppLmpyaWJpQG91dGxvb2suZnIAWwAAI2doYXppLmpyaWJpJW91dGxvb2suZnJAcGFzc3BvcnQuY29tAAABzFROAAAAAAAABAwCAACNY1VAAAZDAAVHaGF6aQAFSnJpYmkAAAAAAAAAAAAAAAAAAAAAAACV1PZXL5uzrQAAvOB1Zc4A32UAAAAAAAAAAAAAAAAQADE5Ni4yMzQuMjQ2LjE0MwAFAwAAAAAAAAAAAAAAAAEAAAAAAAAAAAAAAAAAAAAATCH7am1G/IsAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAADAAAA; xid=e8e465b1-e55f-46ab-891b-980b697fc747&lk_HC&ODSP-ODWEB-ODCF&394; SAToken0=; SAToken1=; wla42=ZHNtMDFwYXAwMDkqMSwyRjlCQjNBRDk1RDRGNjU3LDAsLDAsLTEsLTE=; E=P:KE7pv5n524g=:ffOf3wi2QLDSkwi2y2xNRzkdBXrtrX8GqBr4zDAA/7U=:F; xidseq=13' \
  -H 'Origin: https://onedrive.live.com' \
  -H 'Sec-Fetch-Dest: iframe' \
  -H 'Sec-Fetch-Mode: navigate' \
  -H 'Sec-Fetch-Site: same-site' \
  -H 'Upgrade-Insecure-Requests: 1' \
  -H 'User-Agent: Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Mobile Safari/537.36' \
  -H 'sec-ch-ua: "Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"' \
  -H 'sec-ch-ua-mobile: ?1' \
  -H 'sec-ch-ua-platform: "Android"' \
  --data-raw 'resids=4B518F0277851508%21709&authKey=%21ADJSupKlX_Ij8Yc' \
  --compressed \
  --output "coverage.zip"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  102M    0  102M    0    56   625k      0 --:--:--  0:02:48 --:--:--  113k


In [18]:
path = "/content/coverage/"
if not os.path.exists(path):
      os.makedirs(path)
!unzip -qq "/content/coverage.zip" -d "$path"

***We have donwloaded all the dataset except for NIST2016, because we couldn't find any trace of that dataset in the provided link***

# <font color = "#ffd700" size = 6> Code Inspection and Performance Assessment
We will Examine now and elucidate the implementation of the code. We will Assess the code from various angles, encompassing its implementation simplicity, performance attributes, and its adaptability for future development.

A crucial aspect of ensuring research reproducibility involves evaluating the clarity and efficiency of the authors' code documentation. The objective of this section is to assess whether the code is not only easy to implement but also optimized for execution in less powerful environments.

## `Clone Author's code`

We won't be directly cloning the author's code from their repository due to modifications we've implemented. Instead, we forked the repository, applied the necessary changes, and will clone our dedicated repository for this competition. In this section, we'll provide guidance on the applied modifications to facilitate the implementation of the author's code in our Colab environment.

In [19]:
!git clone https://github.com/Ghazi-jr/CS-challenge.git

Cloning into 'CIS-challenge'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 76 (delta 14), reused 71 (delta 12), pack-reused 0
Receiving objects: 100% (76/76), 54.83 KiB | 3.92 MiB/s, done.
Resolving deltas: 100% (14/14), done.


***We will download too the paths folder, which contains the text files that defines our different train, test, and validation image paths***

In [ ]:
%%time
link = "https://www.dropbox.com/s/opjpz9hoy5xm4um/paths.zip?dl=0"
out_name = "paths.zip"
out_path = "paths"

download_dataset_dropbox(link, out_name, out_path)

**We found out also that the PreciseRolPooling extention isn't updated in the author's code and this will cause some errors in the training of our models so we will clone the Latest Version Of PreciseRolPooling To fix the deprecated implementation of THC/THC.h**

In [ ]:
# @title Correcting THC/THC.h deprecation

!git clone https://github.com/vacancy/PreciseRoIPooling.git
!rm -rf /content/CIS-challenge/utils/lib/nn/prroi_pool
!cp -r /content/PreciseRoIPooling/pytorch/prroi_pool /content/CIS-challenge/utils/lib/nn/

!rm -rf /content/CIS-challenge/utils/lib/nn/prroi_pool/src/prroi_pooling_gpu_impl.cu
!rm -rf /content/CIS-challenge/utils/lib/nn/prroi_pool/src/prroi_pooling_gpu_impl.cuh

!cp /content/PreciseRoIPooling/src/prroi_pooling_gpu_impl.cu /content/CIS-challenge/utils/lib/nn/prroi_pool/src/
!cp /content/PreciseRoIPooling/src/prroi_pooling_gpu_impl.cuh /content/CIS-challenge/utils/lib/nn/prroi_pool/src/

***The files downloaded from the given path contain mappings to images specific to the author's environment. Hence, it is necessary to parse these paths and modify them to align with our environment. Those files are crucial for the training and evaluation of the models.***
<br>
***Again for the purpose of this report we will not include the code of this part since it's quite long but the notebook will contain the detailled implementation of this process***

In [22]:
# @title Defining New Path Mappings

#Old CASIAv2 Mapping
old_casia2_au_image_path = "[ROOT]/ori/CASIAv2//images/Au"
old_casia2_tp_image_path = "[ROOT]/ori/CASIAv2//images/Tp"
old_casia2_tp_masks_path = "[ROOT]/ori/CASIAv2//masks"
old_casia2_tp_edges_path = "[ROOT]/ori/CASIAv2//edges"
#New CASIAv2 Mapping
new_casia2_au_image_path = "/content/casiav2/CASIA2.0_revised/Au"
new_casia2_tp_image_path = "/content/casiav2/CASIA2.0_revised/Tp"
new_casia2_tp_masks_path = "/content/casiav2/CASIA2.0_revised/CASIA2.0_Groundtruth"
new_casia2_tp_edges_path = "/content/casiav2/CASIA2.0_revised/CASIA2.0_Groundtruth"

#Old CASIAv1 Mapping
old_casia1_au_image_path = "[ROOT]/ori/CASIAv1//images/Au"
old_casia1_tp_image_path = "[ROOT]/ori/CASIAv1//images/Modified_Tp"
old_casia1_tp_masks_path = "[ROOT]/ori/CASIAv1//masks"
old_casia1_tp_edges_path = "[ROOT]/ori/CASIAv1//edges"
#New CASIAv1 Mapping
new_casia1_au_image_path = "/content/casiav1/CASIA_1.0_dataset/Au"
new_casia1_tp_image_path = "/content/casiav1/CASIA_1.0_dataset/Tp"
new_casia1_tp_masks_path = "/content/casiav1/CASIA_1.0_dataset/CASIA_1.0_groundtruth"
new_casia1_tp_edges_path = "/content/casiav1/CASIA_1.0_dataset/CASIA_1.0_groundtruth"

#Old Columbia Mapping
old_columbia_au_image_path = "[ROOT]/ori/Columbia//4cam_auth"
old_columbia_tp_image_path = "[ROOT]/ori/Columbia//4cam_splc"
old_columbia_tp_masks_path = "[ROOT]/ori/Columbia//masks"
old_columbia_tp_edges_path = "[ROOT]/ori/Columbia//edges"
#New Columbia Mapping
new_columbia_au_image_path = "/content/colombia/4cam_auth"
new_columbia_tp_image_path = "/content/colombia/4cam_splc"
new_columbia_tp_masks_path = "/content/colombia/masks/edgemask"
new_columbia_tp_edges_path = "/content/colombia/masks/edgemask"

#Old COVERAGE Mapping
old_coverage_au_image_path = "[ROOT]/ori/COVERAGE//image"
old_coverage_tp_image_path = "[ROOT]/ori/COVERAGE//image"
old_coverage_tp_masks_path = "[ROOT]/ori/COVERAGE//mask"
old_coverage_tp_edges_path = "[ROOT]/ori/COVERAGE//edge"

#New COVERAGE Mapping
new_coverage_au_image_path = "/content/coverage/image"
new_coverage_tp_image_path = "/content/coverage/image"
new_coverage_tp_masks_path = "/content/coverage/mask"
new_coverage_tp_edges_path = "/content/coverage/mask"

#DeepFake Mapping
old_deep_path = "[ROOT]"
new_deep_path = "/content/deep-data"

#NIST
to_remove = "NIST2016"

In [ ]:
# @title Parsing Paths

folder_path = '/content/paths/'

def parse_deep_paths(file_path, old_path, new_path) :

    file_path = file_path.replace(old_path, new_path)

    return file_path

def parse_paths(
        file_path,
        old_au_path,
        old_tp_path,
        old_mask_path,
        old_edge_path,
        new_au_path,
        new_tp_path,
        new_mask_path,
        new_edge_path
    ) :

    file_path = file_path.replace(old_au_path, new_au_path)
    file_path = file_path.replace(old_tp_path, new_tp_path)
    file_path = file_path.replace(old_mask_path, new_mask_path)
    file_path = file_path.replace(old_edge_path, new_edge_path)

    return file_path

def modify_line(line, file_name):
    if "CASIAv2" in line :
        line = parse_paths(
            line,
            old_casia2_au_image_path,
            old_casia2_tp_image_path,
            old_casia2_tp_masks_path,
            old_casia2_tp_edges_path,
            new_casia2_au_image_path,
            new_casia2_tp_image_path,
            new_casia2_tp_masks_path,
            new_casia2_tp_edges_path
        )

    elif "CASIAv1" in line :
        line = parse_paths(
            line,
            old_casia1_au_image_path,
            old_casia1_tp_image_path,
            old_casia1_tp_masks_path,
            old_casia1_tp_edges_path,
            new_casia1_au_image_path,
            new_casia1_tp_image_path,
            new_casia1_tp_masks_path,
            new_casia1_tp_edges_path
        )
    elif "Columbia" in line :
        line = parse_paths(
            line,
            old_columbia_au_image_path,
            old_columbia_tp_image_path,
            old_columbia_tp_masks_path,
            old_columbia_tp_edges_path,
            new_columbia_au_image_path,
            new_columbia_tp_image_path,
            new_columbia_tp_masks_path,
            new_columbia_tp_edges_path
        )
    elif "COVERAGE" in line :
        line = parse_paths(
            line,
            old_coverage_au_image_path,
            old_coverage_tp_image_path,
            old_coverage_tp_masks_path,
            old_coverage_tp_edges_path,
            new_coverage_au_image_path,
            new_coverage_tp_image_path,
            new_coverage_tp_masks_path,
            new_coverage_tp_edges_path
        )
    elif "NIST2016" in line :
        line = ""
    elif not "sf" in file_name :
        line = parse_deep_paths(line, old_deep_path, new_deep_path)
    else :
        pass

    return line

file_list = os.listdir(folder_path)
for file_name in file_list:

    file_path = os.path.join(folder_path, file_name)

    if os.path.isfile(file_path):
        print(f"Processing file: {file_name}")
        with open(file_path, 'r') as file:
            lines = file.readlines()

        modified_lines = [modify_line(line, file_name) for line in lines]
        with open(file_path, 'w') as file:
            file.writelines(modified_lines)


***In the remaining of the notebook we will switch to work on the CIS-challenge folder***

In [24]:
%cd CS-challenge/

/content/CIS-challenge


## `Code Inspecting And Assessment`
During this phase, we delved into scrutinizing the author's code, exploring its functionality and assessing its suitability for execution in an environment like Google Colab.

Several key observations and conclusions emerged from our examination, shedding light on the code's accessibility for reproducing the authors' research:

 - Code Readability: The code demonstrated a commendable level of readability, facilitating comprehension. However, we identified room for improvement in terms of documentation and comments. More extensive explanatory notes would enhance understanding of the code's structure and, crucially, the construction of the models. This additional information would be instrumental for anyone seeking to build upon the existing models or experiment with different architectures.

 - Code Performance: Our evaluation extended to the code's performance characteristics, particularly its adaptability to less powerful computing environments. While research settings often boast ample resources, ensuring the code's viability in more modest setups is vital for broader accessibility and reproducibility. Overall, we managed to successfully replicate most of the authors' experiments, yet encountered challenges in certain cases, as elaborated in the Experimenting Section. As part of our optimization considerations, we recognized the authors' adherence to best practices, such as employing multiple workers in their dataloader (Dataloader(dataset, num_workers=4*num_GPU)), utilizing pin_memory (Dataloader(dataset, pin_memory=True)), and implementing DistributedDataParallel. Although various other optimization techniques were utilized, we specifically highlight those related to memory management, a critical aspect, especially in Colab.

 - Update: Notably, the only issue we encountered in the author's code pertained to the PreciseRoIPooling module. We addressed this by updating the code, recognizing that the original implementation had been modified to resolve concerns related to the deprecated THC/THC.h library.










# <font color = "#ffd700" size = 6> Experiments (Author's Work)

- Conduct the Authour's experiments to validate the proposed solution.
- Apply the solution to various datasets for localization in manipulated areas.
- Present results, visualizations, and comparisons with existing methods.

***At this step we were ready to run the authors experiments, In total the author's conducted 9 experiment We will provide in this report the code executed to run the experiments along side the problem we faced to implement that***

The author's used a cluster of 4 powerfull GPUs to run those nine experiments on 50 epochs in a distrubuted manner. However since we didn't have those resources we created a Human cluster. So we created a shared drive to host the results of each experiment and we divided those experiments between us to optimize the running time due to the limitations of using google colab

In [ ]:
#Human Cluster Experiment :)
DRIVE_PATH = "/content/drive/"
USER = "/Shareddrives/CS_CHALLENGE/Experiments/Ghazi/"

drive_checkpoint_path = DRIVE_PATH + USER

from google.colab import drive
drive.mount(DRIVE_PATH, force_remount=True)

Mounted at /content/drive/


As we have access to only one Tesla T4 on Colab, we will disable distributed computing. Consequently, we will adjust the values of `LOCAL_RANK`, `RANK`, and `WORLD_SIZE` in our environment.

## Running Experiments

In [ ]:
import os
os.environ["LOCAL_RANK"] = "0"
os.environ["RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "8888"

### Experiment 1

In [ ]:
#Training Params for experiment 1
train_path_file = "../paths/paths_train_sf.txt"
validation_path_file = "../paths/paths_val_sf.txt"
model = "mvssnet"
image_size = 224
n_epochs = 50
optim = "adam"

In [ ]:
!python -u train_torch.py --paths_file "$train_path_file" --val_paths_file "$validation_path_file" --model "$model" --image_size "$image_size" --n_epochs "$n_epochs" --optim "$optim"

***For this part the checkpoint_name and epoch_model_name are a place holders for each team member to save his experiment in the drive and we run the evaluation script on the best recorded epoch***

In [ ]:
#Evaluation Params
test_path_file = "../paths/paths_test_sf.txt"

checkpoint_name = "" #Update this according to experiment
epoch_model_name = "" #Update this according to best epoch

checkpoint_path = f"/content/CIS-challenge/checkpoints/{checkpoint_name}/{epoch_model_name}"

In [ ]:
!python -u evaluate.py --paths_file "$test_path_file" --load_path "$checkpoint_path" --model "$model" --image_size "$image_size"

In [ ]:
#Persist Checkpoint to drive
checkpoint_path = f"/content/CIS-challenge/checkpoints/{checkpoint_name}"
#Copy Out folder to checkpoint
out_path = "/content/CIS-challenge/out"

!cp -r "$out_path" "$checkpoint_path"
!cp -r "$checkpoint_path" "$drive_checkpoint_path"

***We decided to include only the boiler plate to run one experiment and you could find always in the notebook the full experiment's code***

***Conclusion*** :<br>
Broadly, we successfully executed the majority of experiments employing the authors' specified hyperparameters. However, due to memory constraints and to avert Colab crashes, we had to diminish the image size to 224 from the original 512. Upon running the evaluation script, the outcomes were closely aligned, and the masks generated by different models exhibited significant variations in quality. It is important to acknowledge some deviations in the results, as the training did not precisely utilize the full image quality.

## Helpers For Assessment

***In this section, we aimed to incorporate the foundational code as it constituted the core of our evaluation process. Detailed results will be elaborated upon in the presentation; however, the following code snippet underscores our approach in obtaining experiments from the shared drive. Additionally, we downloaded the authors' experiments from the provided links, enabling us to execute the evaluation script and scrutinize any discernible differences.***

In [ ]:
def load_model_from_drive(drive_path) :
  models_path = "/content/models/"
  if not os.path.exists(models_path):
      os.makedirs(path)
      print(f"Models Path Created !")

  !cp -r "$drive_path" "$models_path"


def download_auth_model(link, file_name, save_path) :

    if not os.path.exists(save_path):
        os.makedirs(save_path)

    full_path = f"{save_path}{file_name}"
    !wget -O  "$full_path" "$link"


DRIVE_PATH = "/content/drive/"
USER = "/Shareddrives/CS_CHALLENGE/Experiments/Ghazi/"
drive_checkpoint_path = DRIVE_PATH + USER

checkpoint_name = ""
epoch_model_name = ""

In [ ]:
drive_path = drive_checkpoint_path + checkpoint_name + "/" + epoch_model_name
load_model_from_drive(drive_path)

In [ ]:
download_auth_model("https://www.dropbox.com/s/zzk4eump5xfbqmz/9.pth?dl=0", "experiment_9.pth", "/content/models/")

In [ ]:
test_path_file = "../paths/paths_test.txt"
checkpoint_path = "/content/models/experiment_9.pth"

!python -u evaluate.py --paths_file "$test_path_file" --load_path "$checkpoint_path" --model "ours" --image_size 512

# <font color = "#ffd700" size = 6> Optimization (Our's Work)
- Explore opportunities for solution optimization.
- Apply various optimization techniques to achieve better results.

In the optimization phase, our approach extended beyond conventional methods. Initially, we evaluated the performance of the authors' code, finding no immediate optimization opportunities, given the robust performance of their architecture and model. While contemplating the implementation of an alternative model, we recognized the challenge of surpassing the efficacy of the state-of-the-art approach developed by the authors.

Prompted by curiosity, we questioned the authors' decision not to segregate DeepFake data from shallowFake data. The rationale became apparent: they aimed to create a model with strong generalization capabilities across both datasets. However, upon scrutinizing their results, we observed that training the model on individual datasets yielded superior outcomes. This led us to a strategic decision – implementing a classification model initially to distinguish between shallow fake and deepfake data. This approach holds real-world significance, considering the unknown nature of incoming images. By leveraging the classifier, we could discern the nature of each instance of fakery and subsequently employ segmentation models trained specifically on either the shallowFake or deepFake datasets.

In [28]:
# @title Classifier Results

%time

import os
import cv2
import numpy as np
from skimage import feature
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import pickle
from tqdm import tqdm

possible_sf_paths = ["/content/casiav2/CASIA2.0_revised/Tp", "/content/casiav1/CASIA_1.0_dataset/Tp/CM", "/content/casiav1/CASIA_1.0_dataset/Tp/Sp", "/content/colombia/4cam_splc"]
possible_df_paths = ["/content/deep-data/image/Deepfakes", "/content/deep-data/image/Face2Face", "/content/deep-data/image/FaceSwap", "/content/deep-data/image/NeuralTextures", "/content/deep-data/image/actors"]
possible_au_paths = ["/content/casiav1/CASIA_1.0_dataset/Au", "/content/casiav2/CASIA2.0_revised/Au", "/content/colombia/4cam_auth", "/content/deep-data/image/youtube"]

def extract_noise_patterns(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    laplacian = cv2.Laplacian(gray_image, cv2.CV_64F)
    laplacian_var = laplacian.var()
    return [laplacian_var]

def extract_compression_artifacts(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    dct_coeffs = cv2.dct(np.float32(gray_image))
    compression_features = dct_coeffs[8:16, 8:16].flatten()
    return compression_features.tolist()

def extract_color_histograms(image):
    hist_b = cv2.calcHist([image], [0], None, [256], [0, 256])
    hist_g = cv2.calcHist([image], [1], None, [256], [0, 256])
    hist_r = cv2.calcHist([image], [2], None, [256], [0, 256])

    color_histogram_features = np.concatenate([hist_b, hist_g, hist_r]).flatten()
    return color_histogram_features.tolist()

def extract_splicing_artifacts(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp = feature.local_binary_pattern(gray_image, P=8, R=1, method="uniform")
    splicing_features = np.histogram(lbp, bins=np.arange(0, 10), density=True)[0]
    return splicing_features.tolist()

def extract_blurring_and_sharpness(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    laplacian_var = cv2.Laplacian(gray_image, cv2.CV_64F).var()
    return [laplacian_var]

def load_images_and_labels(folder_path, label):
    features = []
    labels = []

    for filename in tqdm(os.listdir(folder_path)):
        if filename.endswith(('.jpg', '.jpeg', '.png', '.tif')):
            image_path = os.path.join(folder_path, filename)
            img = cv2.imread(image_path)

            noise_features = extract_noise_patterns(img)
            compression_features = extract_compression_artifacts(img)
            color_histogram_features = extract_color_histograms(img)
            splicing_features = extract_splicing_artifacts(img)
            blurring_sharpness_features = extract_blurring_and_sharpness(img)

            img_features = (
                noise_features +
                compression_features +
                color_histogram_features +
                splicing_features +
                blurring_sharpness_features
            )

            features.append(img_features)
            labels.append(label)

    return features, labels

deepfake_folder = possible_df_paths[0]
shallowfake_folder = possible_sf_paths[0]

deepfake_features, deepfake_labels = load_images_and_labels(deepfake_folder, label='deepfake')
shallowfake_features, shallowfake_labels = load_images_and_labels(shallowfake_folder, label='shallowfake')

for p in possible_df_paths[1:] :
  df_f, df_l = load_images_and_labels(p, label='deepfake')
  deepfake_features.extend(df_f)
  deepfake_labels.extend(df_l)

for p in possible_sf_paths[1:] :
  sf_f, sf_l = load_images_and_labels(p, label='shallowfake')
  shallowfake_features.extend(sf_f)
  shallowfake_labels.extend(sf_l)


all_features = deepfake_features + shallowfake_features
all_labels = deepfake_labels + shallowfake_labels

X = np.array(all_features)
y = np.array(all_labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:')
print(classification_rep)

# save classifier
with open('classifier.pkl','wb') as f:
    pickle.dump(clf,f)

CPU times: user 0 ns, sys: 4 µs, total: 4 µs
Wall time: 7.39 µs


100%|██████████| 5123/5123 [05:24<00:00, 15.78it/s]


Accuracy: 0.99
Classification Report:
              precision    recall  f1-score   support

    deepfake       0.99      0.99      0.99       424
 shallowfake       1.00      1.00      1.00      1001

    accuracy                           0.99      1425
   macro avg       0.99      0.99      0.99      1425
weighted avg       0.99      0.99      0.99      1425



During the classifier training phase, we extracted relevant features from our model. Following thorough research, we identified distinctive characteristics enabling the differentiation between deep fake and shallow fake images, including:

- Noise patterns
- Compression artifacts
- Color histograms
- Splicing artifacts
- Blurring and sharpness

Leveraging these features, a RandomForestModel achieved impressive scores, attaining a 99% accuracy. This outcome instills confidence in the classifier's ability to accurately categorize images. Consequently, we are poised to integrate this classification into the ensemble script, where the segmented models can be employed with assurance.

In [ ]:
test_path_file = "../paths/paths_test.txt"
path_sf = "" #We should add here the path to our best performing model trained on SF data only
path_df = "" #We should add here the path to our best performing model trained on DF data only

In [ ]:
!python -u ensembler.py --paths_file "$test_path_file" --load_path_sf "$path_sf" --load_path_df "$path_df" --model "ours" --image_size 224

***In conclusion, armed with a robust classifier, we are now positioned to optimize each segmentation model independently. This alleviates concerns about introducing bias from other datasets, fostering the prospect of developing a cutting-edge image fakery detection model.***

# <font color = "#ffd700" size = 6> Buisness Model

Given our exceptional approach to detecting image fakery, we are contemplating a startup idea which we aim to succinctly outline through the Business Model Canvas (BMC).